In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize

In [2]:
def fi_DT(x,y):
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(x, y)
    importances = clf.tree_.compute_feature_importances()
    return (-importances).argsort()[:2]

def fi_LDA(x,y):
    clf = LinearDiscriminantAnalysis()
    clf.fit(x, y)
    weights = np.abs(clf.coef_) / np.sum(np.abs(clf.coef_))
    return (-weights[0]).argsort()[:2]

def fi_intersect(x,y):
    return len(np.intersect1d(fi_LDA(x,y), fi_DT(x,y))) / 2.0

In [3]:
parameters = {
    "X": None,
    "y": None,
    "n_experts": 2,
    "iterations": 1,
    "max_depth": 2,
    "init_learning_rate": 100,
    "learning_rate_decay": 1,
    "initialize_with": "random",
    "initialization_method": None,
    "feature_names": None,
    "class_names": None,
    "use_2_dim_gate_based_on": None,
    "use_2_dim_clustering": False,
    "black_box_algorithm": None,
    }

In [4]:
datasets = [
    ["banknote_input.pd","banknote_target.pd"],
    ["adult_input.pd","adult_target.pd"],
    ["bank_input.pd","bank_target.pd"],
    ["breast_cancer_input.np","breast_cancer_target.np"],
    ["hrss_input.pd","hrss_target.pd"],
    ["iris_input.pd","iris_target.pd"],
    ["occupancy_input.pd","occupancy_target.pd"],
    ["pdm6_input.pd","pdm6_target.pd"],
    ["sensorless_input.pd","sensorless_target.pd"],
    ["steel_input.pd","steel_target.pd"],
]

In [ ]:
runs = 100
rows = []
for dataset in datasets:
    data_input = pickle.load(open("../datasets/" + dataset[0], "rb"))
    data_target = pickle.load(open("../datasets/" + dataset[1], "rb"))
    parameters["X"] = data_input
    parameters["y"] = data_target
    modt = MoDT(**parameters)
    intersections = []
    for _ in range(runs):
        intersections.append(fi_intersect(modt.X,modt.y))
    intersection = np.sum(intersections) / runs
    dict1 = {
        "dataset" : dataset[0],
        "n_features" : modt.X.shape[1],
        "intersection" : intersection,
    }
    rows.append(dict1)
df = pd.DataFrame(rows)
df